In [2]:
import torch
import torch.nn as nn

Батч нормализация работает вдоль канала, то есть если у нас тензор размера ЧИСЛО_КАРТИНОК, ЧИСЛО_КАНАЛОВ, ВЫСОТА, ШИРИНА, то считаем среднее и дисперсию для каждого канала (То есть их размер будет 1, ЧИСЛО_КАНАЛОВ, 1, 1). Усредняем по всему батчу каждый канал Нормализация по слою работает по другому, она нормализует по любому количеству последних размерностей. То есть если у нас текст и нам надо нормализовать эмбеддинги, то нормализуем по их размерности. Вход будет ЧИСЛО_ТЕКСТОВ, ДЛИНА_ТЕКСТА, РАЗМЕР_ЭМБЕДДИНГА, то посчитав среднее и дисперсию получим (ЧИСЛО_ТЕКСТОВ, ДЛИНА_ТЕКСТА, 1), то есть среднее и дисперсию для каждого токена каждого текста в батче, считаем только по последней размерности.
Но нам ничего не мешает взять и посчитать по другому количеству последних осей, например по двум, то есть среднее будет посчитано как (ЧИСЛО ТЕКСТОВ, 1, 1), то есть усредняем каждый текст

In [3]:
# batchnorm
batch_size, channels_num, w, h = 32, 3, 15, 12

input = torch.randn(batch_size, channels_num, w, h)
batchnorm = nn.BatchNorm2d(channels_num)

print('Input:', input)

output_batchnorm = batchnorm(input)
print('Output torch:', output_batchnorm)

mean = torch.mean(input, dim=(0, 2, 3), keepdims=True)
print('Mean shape:', mean.shape)

std = torch.std(input, dim=(0, 2, 3), keepdims=True, correction=0)
print('Std shape:', std.shape)

eps = 1e-5

y_custom = (input - mean) / (std+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_batchnorm, y_custom), 'Tensors do not match.'

Input: tensor([[[[-8.1852e-01, -6.5687e-02,  3.7577e-01,  ...,  1.5342e+00,
            1.0519e+00, -1.7811e+00],
          [-4.7268e-01, -1.2895e+00,  1.5329e-02,  ..., -3.0442e+00,
            7.4050e-02, -3.2899e-01],
          [-1.5758e+00, -3.8819e-01, -3.5753e-01,  ...,  5.1938e-01,
            7.6506e-01,  3.5995e-01],
          ...,
          [-1.8833e-02,  6.0201e-01,  9.3248e-02,  ...,  1.6997e+00,
            2.4579e-01,  6.9284e-01],
          [ 1.4979e+00, -3.4515e-01, -1.8582e+00,  ...,  7.7811e-01,
           -8.2529e-01, -2.7332e-01],
          [ 7.6027e-01,  1.1731e-02,  1.6514e+00,  ..., -1.8339e+00,
           -1.7353e+00, -1.2431e-01]],

         [[-2.5683e-01,  6.5593e-02, -4.2694e-01,  ...,  5.6948e-01,
           -1.6648e+00,  7.1374e-01],
          [ 1.6944e+00, -5.3848e-02,  8.2776e-01,  ...,  1.2921e-01,
            8.5289e-01,  6.6426e-01],
          [ 1.7289e+00,  1.0044e+00,  6.7332e-01,  ...,  1.3578e+00,
            1.3801e-01,  1.0089e+00],
          ...

In [4]:
# batchnorm1d, тоже самое, по каналам
batch_size, channels_num, w = 32, 3, 15

input = torch.randn(batch_size, channels_num, w)
batchnorm = nn.BatchNorm1d(channels_num)

print('Input:', input)

output_batchnorm = batchnorm(input)
print('Output torch:', output_batchnorm)

mean = torch.mean(input, dim=(0, 2), keepdims=True)
print('Mean shape:', mean.shape)

std = torch.std(input, dim=(0, 2), keepdims=True, correction=0)
print('Std shape:', std.shape)

eps = 1e-5

y_custom = (input - mean) / (std+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_batchnorm, y_custom), 'Tensors do not match.'

Input: tensor([[[ 1.4919, -0.5560,  0.8890,  ..., -1.3067, -0.7609,  0.0973],
         [-1.2144,  0.6094, -0.1687,  ...,  0.0102,  0.4285, -1.0172],
         [-0.5431, -1.6942,  0.7371,  ...,  0.5064, -1.5739,  1.0819]],

        [[-1.9588,  0.6448,  0.7925,  ...,  0.8270,  1.1089,  0.5213],
         [-0.6715, -0.0580,  0.0617,  ...,  0.4142, -1.9328,  0.0038],
         [ 0.9281, -1.9258,  0.4892,  ..., -1.2111, -2.0832,  0.2553]],

        [[ 1.2151,  0.4981, -0.5987,  ...,  0.0405, -0.8314, -0.5856],
         [ 0.8436,  0.1286, -0.1122,  ...,  0.4449,  1.3537,  1.5335],
         [ 0.9474, -0.2004,  0.8246,  ..., -1.5710, -0.1749,  0.7695]],

        ...,

        [[ 0.3169, -0.1831,  1.8064,  ..., -0.4885, -0.3419,  0.4158],
         [-0.9033,  1.1457, -0.0849,  ..., -1.0375,  2.0772, -0.4704],
         [-0.1002,  1.7049,  0.5509,  ...,  1.5785, -0.9180, -2.7936]],

        [[-1.1461,  3.1468,  0.6714,  ...,  0.4698,  0.9895, -0.9102],
         [ 0.4388,  2.3494, -0.5159,  ..., -0.48

In [10]:
# layernorm
batch_size, seq_len, emb_size = 32, 30, 16

input = torch.randn(batch_size, seq_len, emb_size)
layernorm = nn.LayerNorm(emb_size, elementwise_affine=False, bias=False)

print('Input:', input)

output_layernorm = layernorm(input)
print('Output torch:', output_layernorm)

mean = torch.mean(input, dim=(2), keepdims=True) # считаем среднее по каждому элементу батча и каждому "токену"
print('Mean shape:', mean.shape)

std = torch.std(input, dim=(2), keepdims=True, correction=0)
print('Std shape:', std.shape)

eps = 1e-5

y_custom = (input - mean) / (std+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_layernorm, y_custom, atol=1e-7), 'Tensors do not match.'

Input: tensor([[[ 1.2001, -0.3311, -0.4078,  ..., -0.0951,  0.5433, -0.4825],
         [-0.1266, -0.0605,  0.3652,  ..., -1.9341,  0.9664, -0.5786],
         [-0.3651,  0.4247, -0.7281,  ..., -0.1626, -1.2925,  0.8041],
         ...,
         [ 0.9020, -0.6529,  1.3645,  ..., -0.5095, -1.4092,  1.0656],
         [-0.7960, -0.5152, -0.2504,  ..., -1.1004, -0.9493,  1.4845],
         [-1.9965,  0.3834, -0.3535,  ...,  1.7050,  0.3811,  0.1960]],

        [[-0.3569,  0.6537,  0.9945,  ..., -0.4525, -0.4234, -1.6739],
         [-0.7635,  1.5416, -0.1292,  ..., -1.1454,  0.2506, -0.8453],
         [-0.9962, -0.3108, -0.4811,  ...,  0.3184, -0.2225,  0.0166],
         ...,
         [ 0.1823,  0.4190, -0.2829,  ...,  0.5321,  1.3356, -0.9623],
         [ 1.7486,  0.0171,  0.6897,  ...,  0.4588,  0.0653,  0.0369],
         [-0.2976, -2.1883, -1.8556,  ..., -0.0677, -0.2267, -0.0103]],

        [[-0.5847,  0.9820,  1.2353,  ...,  1.4064,  0.8490,  0.8148],
         [ 0.1307, -0.6377,  0.8212,  

In [11]:
# layernorm, можно взять произвольное количество последних осей
batch_size, seq_len, emb_size = 32, 30, 16

input = torch.randn(batch_size, seq_len, emb_size)
layernorm = nn.LayerNorm((seq_len, emb_size), elementwise_affine=False, bias=False)

print('Input:', input)

output_layernorm = layernorm(input)
print('Output torch:', output_layernorm)

mean = torch.mean(input, dim=(1, 2), keepdims=True) # считаем среднее по каждому элементу батча и каждому "токену"
print('Mean shape:', mean.shape)

std = torch.std(input, dim=(1, 2), keepdims=True, correction=0)
print('Std shape:', std.shape)

eps = 1e-5

y_custom = (input - mean) / (std+eps)
print("y_custom:", y_custom)
assert torch.allclose(output_layernorm, y_custom, atol=1e-7), 'Tensors do not match.'

Input: tensor([[[ 1.1155e+00, -1.5700e+00, -8.3913e-01,  ...,  1.5184e+00,
          -6.6980e-01, -3.1623e-01],
         [-1.2693e+00,  6.4082e-01, -7.4249e-01,  ..., -1.4555e+00,
           8.6159e-01, -5.5728e-02],
         [-2.3085e-01,  8.0145e-01, -1.4131e+00,  ..., -1.0859e+00,
          -1.0465e+00,  7.8524e-01],
         ...,
         [ 2.6613e-01,  1.3807e+00, -1.8532e-01,  ..., -1.3601e+00,
          -1.7471e+00,  9.2451e-01],
         [-8.1278e-01,  8.6840e-01, -1.5942e+00,  ..., -2.8124e+00,
           5.3697e-01, -1.0354e+00],
         [ 3.7199e-02,  1.5979e+00, -1.1902e+00,  ...,  2.8849e-01,
          -6.0523e-01,  1.4875e+00]],

        [[-2.5899e-02,  4.6765e-01, -1.5687e-01,  ...,  1.1428e-01,
           1.3832e-01,  7.2878e-01],
         [ 3.3943e-01, -4.1848e-01, -1.6186e-01,  ...,  1.6119e-01,
           2.0039e+00,  1.1236e+00],
         [-1.2891e+00, -2.5828e-03,  2.7354e-02,  ...,  1.0633e+00,
          -2.4753e-03, -2.8415e-01],
         ...,
         [-4.8190e